load features

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from src.config import INTERIM_DIR

Zp_train = np.load(INTERIM_DIR / "Zp_train.npy")
Za_train = np.load(INTERIM_DIR / "Za_train.npy")


PCA Eigenvalue scree plot

In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(np.var(Zp_train, axis=0), marker="o")
plt.title("PCA Latent Variance Spectrum")
plt.xlabel("Component Index")
plt.ylabel("Variance")
plt.grid()
plt.show()


2D Latent Projections (PCA vs. AE)

In [3]:
fig, ax = plt.subplots(1, 2, figsize=(12, 5))

ax[0].scatter(Zp_train[:, 0], Zp_train[:, 1], s=5)
ax[0].set_title("PCA Latent Space")

ax[1].scatter(Za_train[:, 0], Za_train[:, 1], s=5)
ax[1].set_title("Autoencoder Latent Space")

plt.show()


NameError: name 'plt' is not defined

Reconstruction Comparison

In [4]:
X_train = np.load(INTERIM_DIR / "X_train.npy")

from src.features.pca_svd import PCAFeatureExtractor
from src.features.autoencoder import AutoencoderFeatureExtractor

pca = PCAFeatureExtractor(n_components=Zp_train.shape[1])
Za = AutoencoderFeatureExtractor(latent_dim=Za_train.shape[1])

x = X_train[0]

x_pca = pca.inverse_transform(pca.transform(x.reshape(1, -1)))[0]
x_ae = Za.reconstruct(x.reshape(1, -1))[0]

fig, ax = plt.subplots(1, 3, figsize=(12, 4))
ax[0].plot(x[:50]); ax[0].set_title("Original")
ax[1].plot(x_pca[:50]); ax[1].set_title("PCA Recon")
ax[2].plot(x_ae[:50]); ax[2].set_title("AE Recon")
plt.show()


NameError: name 'np' is not defined